TEXT PROCESSING DATA TWEET

In [1]:
#Menginmport Library Untuk s Processing

import pandas as pd # library untuk data manipulation
import re
import sqlite3

In [2]:
#Menginmport data tweet

df = pd.read_csv('data.csv', encoding='latin-1')
df


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,USER jangan asal ngomong ndasmu. congor lu yg ...,1,1,1,0,0,0,1,0,0,1,0,0
13165,USER Kasur mana enak kunyuk',0,1,0,0,0,0,0,0,0,0,0,0
13166,USER Hati hati bisu :( .g\n\nlagi bosan huft \...,0,0,0,0,0,0,0,0,0,0,0,0
13167,USER USER USER USER Bom yang real mudah terdet...,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
df.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


Mengolah Data Abusive   

In [4]:
# Mengubah kata ke huruf kecil

def lowercase(s):
    return s.lower()


In [5]:
#Menghapus tanda baca

def punctuation(s):
    s = re.sub('[^0-9a-zA-Z]+', ' ', s) #menghilangkan semua karakter yang bukan huruf atau angka dan menggantinya dengan spasi.
    s = re.sub('^rt',' ', s) #menghapus awalan rt
    s = re.sub('gue','saya', s) # Mengganti kata "gue" dengan kata "saya"
    s = re.sub(r'\d+', '', s) #menghapus semua angka
    s = re.sub('user',' ', s) #menghapus kata 'user'
    s = re.sub(r':', ' ', s) #menggantikan karakter : dengan spasi
    s = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ', s) #menghapus semua URL 
    s = re.sub(' +', ' ', s) #menggantikan satu atau lebih spasi berturut-turut dengan satu spasi 
    s = re.sub('\n',' ',s) #menggantikan karakter newline (\n) dengan spasi 
    s = re.sub(r'pic.twitter.com.[\w]+', ' ', s) #menghapus semua tautan Twitter (pic.twitter.com)
    return s

In [6]:
# Membuat Koneksi ke database
conn = sqlite3.connect('database_abusive.db')

In [7]:
# Import Abusive
abusive = pd.read_csv('abusive.csv')

# Konek Abusive ke SQL
abusive.to_sql('abusive', conn, if_exists='replace', index=False)
abusive=pd.DataFrame(abusive)
abusive

,ABUSIVE
0,alay
1,ampas
2,buta
3,keparat
4,anjing
...,...
120,rezim
121,sange
122,serbet
123,sipit


In [8]:
# Mengambil Kembali Abusive

db = sqlite3.connect('database_abusive.db', check_same_thread=False)
q_abusive = 'SELECT * FROM abusive'
t_abusive = pd.read_sql_query(q_abusive, db)

In [9]:
# Fungsi Hapus Kata Abusive

def remove_abusive_words(s):
    abusive_words = set(t_abusive['ABUSIVE'])
    words = s.split()
    filtered_words = [word for word in words if word not in abusive_words]
    return ' '.join(filtered_words)

Mengolah data Kamus Alay

In [10]:
#Membuat Koneksi ke database
conn = sqlite3.connect('database_alay.db')

In [11]:
#Import Kamusalay
kamusalay = pd.read_csv('new_kamusalay.csv',encoding='latin-1',names=['alay','normal'])

#Konek Kamus alay ke database

kamusalay.to_sql('kamusalay', conn, if_exists='replace',index=False)
kamusalay=pd.DataFrame(kamusalay)
kamusalay

,alay,normal
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali
...,...,...
15162,mendikbud,menteri pendidikan dan kebudayaan
15163,mendag,menteri perdagangan
15164,menaker,menteri tenaga kerja
15165,memetwit,mentwit


In [12]:
#Mengambil Kembali kamusalay

db = sqlite3.connect('database_alay.db', check_same_thread = False)
q_kamusalay = 'SELECT * FROM kamusalay'
t_kamusalay = pd.read_sql_query(q_kamusalay, db)

In [13]:
# Fungsi Replace Kata Alay ke Kata Normal

alay_dict = dict(zip(t_kamusalay['alay'], t_kamusalay['normal']))
def alay_to_normal(s):
    for word in alay_dict:
        return ' '.join([alay_dict[word] if word in alay_dict else word for word in s.split(' ')])

Menerapkan Funsgi Text Processing dan Cleansing

In [14]:
# Copy Dataset

data = df.copy()
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [15]:
#Membuat Fungsi Text Cleansing

def cleansing(s):
    s = lowercase(s)
    s = punctuation(s)
    s = remove_abusive_words(s)
    s = alay_to_normal(s)
    return s

In [16]:
data['Tweet'] = data['Tweet'].apply(cleansing)
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,di saat semua cowok berusaha melacak perhatian...,1,1,1,0,0,0,0,0,1,1,0,0
1,siapa yang telat memberi tau kamu saya bergaul...,0,1,0,0,0,0,0,0,0,0,0,0
2,kadang aku berpikir kenapa aku tetap percaya p...,0,0,0,0,0,0,0,0,0,0,0,0
3,aku itu aku dan ku tau matamu tapi dilihat dar...,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum kafir sudah kelihatan dongoknya dari awal...,1,1,0,1,1,0,0,0,0,0,1,0


In [17]:
#Membandingkan Tweet lama dan Tweet yang sudah di Cleansing


print('Tweet lama: {}'.format(df['Tweet'].iloc[3]))
print('\t')
print('Tweet baru: {}'.format(data['Tweet'].iloc[3]))

Tweet lama: USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT TAPI DILIAT DARI MANA ITU AKU'
	
Tweet baru: aku itu aku dan ku tau matamu tapi dilihat dari mana itu aku


Menyiman data yang sudah dibersihkan

In [18]:
#Koneksi data yang sudah dibersihkan ke Database

data.to_sql('data', conn, if_exists='replace',index=False)
data=pd.DataFrame(data)
data

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,di saat semua cowok berusaha melacak perhatian...,1,1,1,0,0,0,0,0,1,1,0,0
1,siapa yang telat memberi tau kamu saya bergaul...,0,1,0,0,0,0,0,0,0,0,0,0
2,kadang aku berpikir kenapa aku tetap percaya p...,0,0,0,0,0,0,0,0,0,0,0,0
3,aku itu aku dan ku tau matamu tapi dilihat dar...,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum kafir sudah kelihatan dongoknya dari awal...,1,1,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,jangan asal berbicara ndasmu kamu yang sekate ...,1,1,1,0,0,0,1,0,0,1,0,0
13165,kasur mana enak,0,1,0,0,0,0,0,0,0,0,0,0
13166,hati hati tidak dan lagi bosan duh xf xf x xaa,0,0,0,0,0,0,0,0,0,0,0,0
13167,bom yang real mudah terdeteksi bom yang terkub...,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
#Menutup Database
conn.commit()
conn.close()

In [20]:
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,di saat semua cowok berusaha melacak perhatian...,1,1,1,0,0,0,0,0,1,1,0,0
1,siapa yang telat memberi tau kamu saya bergaul...,0,1,0,0,0,0,0,0,0,0,0,0
2,kadang aku berpikir kenapa aku tetap percaya p...,0,0,0,0,0,0,0,0,0,0,0,0
3,aku itu aku dan ku tau matamu tapi dilihat dar...,0,0,0,0,0,0,0,0,0,0,0,0
4,kaum kafir sudah kelihatan dongoknya dari awal...,1,1,0,1,1,0,0,0,0,0,1,0
